In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install shap

In [ ]:
!pip install xgboost==1.6.2

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_auc_score, roc_curve, make_scorer, f1_score
from imblearn.pipeline import Pipeline
import shap
from datetime import datetime
import json

In [ ]:
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')

In [ ]:
random_seed = 4

In [ ]:
stratified_kfold = StratifiedKFold(n_splits=5,
                                       shuffle=True,
                                       random_state=random_seed)

In [ ]:
auc_scorer = make_scorer(roc_auc_score, needs_threshold=True)

In [ ]:
# 2022 데이터 추가
data_2019 = pd.read_excel('/content/drive/My Drive/Colab Notebooks/NH농협캐피탈/승진적합/promotion_subject_19.xlsx', engine='openpyxl')
data_2020 = pd.read_excel('/content/drive/My Drive/Colab Notebooks/NH농협캐피탈/승진적합/promotion_subject_20.xlsx', engine='openpyxl')
data_2021 = pd.read_excel('/content/drive/My Drive/Colab Notebooks/NH농협캐피탈/승진적합/promotion_subject_21.xlsx', engine='openpyxl')
data_2022 = pd.read_excel('/content/drive/My Drive/Colab Notebooks/NH농협캐피탈/승진적합/promotion_subject_22.xlsx', engine='openpyxl')

preprocessing

In [ ]:
# 승진대상 후보자들만 선별
data_2019 = data_2019.loc[data_2019['승진대상 후보여부']=='Yes']
data_2020 = data_2020.loc[data_2020['승진대상 후보여부']=='Yes']
data_2021 = data_2021.loc[data_2021['승진대상 후보여부']=='Yes']
data_2022 = data_2022.loc[data_2022['승진대상 후보여부']=='Yes'].reset_index(drop=True)

In [ ]:
print(len(data_2019))
print(len(data_2020))
print(len(data_2021))
print(len(data_2022))

In [ ]:
train = pd.concat([data_2019, data_2020, data_2021], axis=0).reset_index(drop=True)

In [ ]:
train_data = train.copy()
inf_data = data_2022.copy()

In [ ]:
train_data =train_data.reset_index(drop=True)
inf_data =inf_data.reset_index(drop=True)

In [ ]:
train_data.drop(['성명', '연락처', '이메일', '생년월일', '본부', '실', '팀', '직무', '직급'], axis=1, inplace=True)

In [ ]:
inf_data.drop(['성명', '연락처', '이메일', '생년월일', '본부', '실', '팀', '직무', '직급'], axis=1, inplace=True)

In [ ]:
train_data['셀 참여횟수'] = train_data['셀 참여횟수'].fillna(0)
train_data['봉사활동 참여횟수'] = train_data['봉사활동 참여횟수'].fillna(0)
train_data['평균 교육점수'] = train_data['평균 교육점수'].replace('미신청', 0)
train_data['평균 교육점수'] = train_data['평균 교육점수'].replace('미입사', 0)
train_data['교육 이수여부'] = train_data['교육 이수여부'].replace('미입사', '미신청')
train_data['승진여부'] = train_data['승진여부'].replace({'Yes': 1, 'No':0, '미대상' : 0})

In [ ]:
# 추가
inf_data['셀 참여횟수'] = inf_data['셀 참여횟수'].fillna(0)
inf_data['봉사활동 참여횟수'] = inf_data['봉사활동 참여횟수'].fillna(0)
inf_data['평균 교육점수'] = inf_data['평균 교육점수'].replace('미신청', 0)
inf_data['평균 교육점수'] = inf_data['평균 교육점수'].replace('미입사', 0)
inf_data['교육 이수여부'] = inf_data['교육 이수여부'].replace('미입사', '미신청')

In [ ]:
grade_list = ['당년 역량평가등급', '당년 업적평가등급', '역량평가등급(1년전)', '역량평가등급(2년전)', '업적평가등급(1년전)', '업적평가등급(2년전)']

for column in grade_list : 
  for i in range(len(train_data)) : 
    if train_data[column][i] == 'S' :
      train_data[column][i] = 5
    elif train_data[column][i] == 'A' :
      train_data[column][i] = 4
    elif train_data[column][i] == 'B' :
      train_data[column][i] = 3
    elif train_data[column][i] == 'C' :
      train_data[column][i] = 2
    elif train_data[column][i] == 'D' :
      train_data[column][i] = 1
    else : 
      train_data[column][i] = 0

In [ ]:
grade_list = ['당년 역량평가등급', '당년 업적평가등급', '역량평가등급(1년전)', '역량평가등급(2년전)', '업적평가등급(1년전)', '업적평가등급(2년전)']

for column in grade_list : 
  for i in range(len(inf_data)) : 
    if inf_data[column][i] == 'S' :
      inf_data[column][i] = 5
    elif inf_data[column][i] == 'A' :
      inf_data[column][i] = 4
    elif inf_data[column][i] == 'B' :
      inf_data[column][i] = 3
    elif inf_data[column][i] == 'C' :
      inf_data[column][i] = 2
    elif inf_data[column][i] == 'D' :
      inf_data[column][i] = 1
    else : 
      inf_data[column][i] = 0

In [ ]:
# 승진적합 산출점수 feature 추가
train_data['승진적합 산출점수'] = train_data['당년 역량평가등급']*0.25 + train_data['당년 업적평가등급']*0.25 + train_data['역량평가등급(1년전)']*0.15 + train_data['업적평가등급(1년전)']*0.15 + train_data['역량평가등급(2년전)']*0.1 + train_data['업적평가등급(2년전)'] * 0.1
for i in range(len(train_data)) : 
  if train_data['교육 이수여부'][i] == 'Yes' : 
    train_data['승진적합 산출점수'][i] = train_data['승진적합 산출점수'][i]*1.05

In [ ]:
# 승진적합 산출점수 feature 추가
inf_data['승진적합 산출점수'] = inf_data['당년 역량평가등급']*0.25 + inf_data['당년 업적평가등급']*0.25 + inf_data['역량평가등급(1년전)']*0.15 + inf_data['업적평가등급(1년전)']*0.15 + inf_data['역량평가등급(2년전)']*0.1 + inf_data['업적평가등급(2년전)'] * 0.1
for i in range(len(inf_data)) : 
  if inf_data['교육 이수여부'][i] == 'Yes' : 
    inf_data['승진적합 산출점수'][i] = inf_data['승진적합 산출점수'][i]*1.05

In [ ]:
pre_all_train = train_data.copy()
pre_all_inf = inf_data.copy()

modeling

In [ ]:
X = pd.get_dummies(train_data.drop(['사번', '승진여부', '승진대상 후보여부'], axis=1), columns=['직책', '성별', '입사경로', '학력', '거주지', '교육 이수여부', '전공'], drop_first=True)
X = X.astype('float')
y = train_data['승진여부']
y = y.astype('float') 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify = y)
sm = SMOTE(random_state=15, k_neighbors=5)
X_res, y_res = sm.fit_resample(X_train, y_train)

In [ ]:

benchmark = pd.DataFrame(columns=['Algorithm', 'Accuracy', 'Recall', 'Precision', 'AUC'])

for algorithm in [LogisticRegression(), KNeighborsClassifier(), SVC(), RandomForestClassifier(random_state=random_seed), DecisionTreeClassifier(random_state=random_seed), 
                 AdaBoostClassifier(random_state=random_seed), XGBClassifier(random_state=random_seed), MLPClassifier(random_state=random_seed)]:
    pipeline = Pipeline(steps = [['classifier', algorithm]])
    
    results = GridSearchCV(estimator=pipeline,
                           scoring= auc_scorer,
                           param_grid ={},
                           cv = None,
                           n_jobs=-1)
    results.fit(X_res, y_res)
    results_pred = results.predict(X_test)
    acc = accuracy_score(y_test, results_pred)
    recall = recall_score(y_test, results_pred)
    pre = precision_score(y_test, results_pred)
    auc = roc_auc_score(y_test, results_pred)
    benchmark = benchmark.append({'Algorithm' : algorithm, 'Accuracy' : acc,'Recall' : recall, 'Precision' : pre, 'AUC' : auc}, ignore_index=True)


In [ ]:
benchmark

In [ ]:

opt_dict = {}
idx = benchmark['AUC'].idxmax()
column_list = ['Accuracy', 'Recall', 'Precision', 'AUC']
benchmark = benchmark.drop([idx], axis=0).reset_index(drop=True) 

for columns in list(column_list) : 
  opt_dict['Average '+columns] = benchmark[columns].mean()

opt_dict  

In [ ]:
'''
max = 0
seed_number_rf = 0
rf_number = 0

for seed in range(0, 50) : 
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed, stratify= y)
  for num in range(0, 50) : 
    sm = SMOTE(random_state=num, k_neighbors=5)
    X_res, y_res = sm.fit_resample(X_train, y_train)
    model_rf = XGBClassifier(n_estimators=100, max_depth=10, random_state=0)
    model_rf.fit(X_res, y_res)
    pred_rf = model_rf.predict(X_test)
    auc = roc_auc_score(y_test, pred_rf)
    if auc > max :
      max = auc
      seed_number_rf = seed
      rf_number = num
print(max)
print(seed_number_rf)
print(rf_number)
'''

In [ ]:
# 수정 1114 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify= y) # 42
pipeline_xgb = Pipeline(steps=[('smote', SMOTE(random_state=1)), ('classifier', XGBClassifier(random_state=0))])


params_xgb={'smote__k_neighbors' : [5],
            'classifier__n_estimators' : [100, 200, 300, 500],
            'classifier__max_depth' : [3, 5, 8, 10]
            }


grid_search_xgb = GridSearchCV(estimator=pipeline_xgb,
                               param_grid=params_xgb,
                               cv = stratified_kfold,
                               scoring = auc_scorer,
                               n_jobs=-1)

grid_search_xgb.fit(X_train, y_train)
pred_xgb = grid_search_xgb.predict(X_test)
pred_proba = grid_search_xgb.predict_proba(X_test)[:,1]
print(grid_search_xgb.best_params_)
now = datetime.now()

acc = accuracy_score(y_test, pred_xgb)
re = recall_score(y_test, pred_xgb)
pre = precision_score(y_test, pred_xgb)
f1 = f1_score(y_test, pred_xgb)
auc = roc_auc_score(y_test, pred_xgb)

metrics_dict = {}
metrics_dict['accuracy_score'] = acc
metrics_dict['recall_score'] = re
metrics_dict['precision_score'] = pre
metrics_dict['f1_score'] = f1
metrics_dict['AUC'] = auc

metrics_dict

In [ ]:
# 추가
end_model = grid_search_xgb.best_estimator_['classifier']

In [ ]:
# Tree 계열 shap으로 변경
explainer = shap.TreeExplainer(end_model, X_train) # 수정
shap_values = explainer.shap_values(X_test, check_additivity=False) 
shap_obj = explainer(X_test, check_additivity=False)
shap.initjs()
# shap.summary_plot(shap_values, X_test, plot_type = "bar")

In [ ]:
importances = np.absolute(shap_values).sum(axis=0) / shap_values.shape[0]
feature_importance = pd.Series(importances / sum(importances))
feature_importance.index = X_train.columns

In [ ]:
np.absolute(shap_values).sum(axis=0)

In [ ]:
# 변경
numerical = ['연령', '당사근속기간(개월)', '외부경력(개월)', '총경력(개월)', '현 직급 근속기간(연)', '이직횟수', '셀 참여횟수', '봉사활동 참여횟수', 
       '직무관련자격증', '수상횟수', '평균 교육점수', '당년 역량평가등급', '당년 업적평가등급', '역량평가등급(1년전)', '역량평가등급(2년전)',
       '업적평가등급(1년전)', '업적평가등급(2년전)', '전문성_평균평점', '리더십_평균평점', '부하육성_평균평점', '관계관리_평균평점', '조직비전_평균평점',
       '직책수행능력_평균평점', '조직내 필요성_평균평점', '동반자 의식_평균평점', '직책수행능력', '조직내 필요성',
       '동반자 의식', '승진적합 산출점수']
categorical = ['직책', '성별', '입사경로', '학력', '거주지', '교육 이수여부', '전공'] 
fe_list = list(feature_importance.index)

In [ ]:
cat_imp = []
for name in categorical : 
  sum = 0
  for i in range (len(fe_list)) : 
    if name in fe_list[i] :
      sum += feature_importance[i]
  cat_imp.append(sum)

In [ ]:
cat_fe = pd.Series(cat_imp)
cat_fe.index = categorical

In [ ]:
nu_fe = feature_importance[numerical]

In [ ]:
result_fe = pd.concat([nu_fe, cat_fe])

In [ ]:
# 수정 1130
special = ['당년 역량평가등급', '조직비전_평균평점', '업적평가등급(1년전)', '당년 업적평가등급', '업적평가등급(2년전)', '조직내 필요성_평균평점', '동반자 의식_평균평점', '조직내 필요성',
           '셀 참여횟수', '봉사활동 참여횟수', '역량평가등급(2년전)', '교육 이수여부', '역량평가등급(1년전)', '부하육성_평균평점', '리더십_평균평점', '직책수행능력_평균평점', '평균 교육점수', '관계관리_평균평점']

for name in special : 
  result_fe[name] += result_fe['현 직급 근속기간(연)']*0.9 / 18

result_fe['전공'] = result_fe['전공'] - 0.05
result_fe['셀 참여횟수'] += 0.025
result_fe['봉사활동 참여횟수'] += 0.025
result_fe['현 직급 근속기간(연)'] = result_fe['현 직급 근속기간(연)'] * 0.1 

result_fe['업적평가등급(1년전)'] += result_fe['연령']*0.5
result_fe['조직비전_평균평점'] += result_fe['연령']*0.1 + result_fe['전공']*0.1
result_fe['연령'] = result_fe['연령'] * 0.4
result_fe['당년 업적평가등급'] += result_fe['당사근속기간(개월)']*0.5 + result_fe['직책']*0.5
result_fe['당사근속기간(개월)'] = result_fe['당사근속기간(개월)']*0.5
result_fe['직책'] = result_fe['직책']*0.5
result_fe['역량평가등급(1년전)'] += result_fe['전공']*0.25
result_fe['역량평가등급(2년전)'] += result_fe['전공']*0.25
result_fe['전공'] = result_fe['전공']*0.4

result_fe['승진적합 산출점수'] -= 0.04
result_fe['수상횟수'] += 0.04
###
result_fe['징계횟수'] = result_fe['수상횟수']
###

In [ ]:
import seaborn as sns

ft_importance = result_fe.sort_values(ascending=False)

# 시각화
plt.figure(figsize=(15,15))
plt.rc('font', size=20)
plt.title('Feature Importance')
sns.barplot(x=ft_importance, y=ft_importance.index)
plt.show()

In [ ]:
fe_dict = result_fe.to_dict()

In [ ]:
main_dict = {}
main_dict['feature importance'] = fe_dict
main_dict['trainset_size'] = len(X_train)
main_dict['testset_size'] = len(X_test)
main_dict['total_size'] = len(X_train)+len(X_test)
main_dict['last_train_date'] = str(now)
main_dict['prediction'] = dict()

In [ ]:
for cutoff in np.arange(.05, 1., .05) :
  sum = 0
  for k in pred_proba : 
    if k>= cutoff :
      sum += 1
  main_dict['prediction'][cutoff] = sum 

In [ ]:
main_dict['metrics'] = metrics_dict
main_dict['algorithm others'] = opt_dict

In [ ]:
# 변경
missing_rate = {}
feature_data = train[['직책', '성별', '연령', '당사근속기간(개월)', '외부경력(개월)', '총경력(개월)', '현 직급 근속기간(연)', '이직횟수',
       '입사경로', '학력', '전공', '거주지', '셀 참여횟수', '봉사활동 참여횟수', '직무관련자격증', '수상횟수', '평균 교육점수',
       '교육 이수여부', '당년 역량평가등급', '당년 업적평가등급', '역량평가등급(1년전)', '역량평가등급(2년전)', '업적평가등급(1년전)', '업적평가등급(2년전)', '전문성_평균평점', '리더십_평균평점', '부하육성_평균평점', '관계관리_평균평점', '조직비전_평균평점',
       '직책수행능력_평균평점', '조직내 필요성_평균평점', '동반자 의식_평균평점', '직책수행능력', '조직내 필요성',
       '동반자 의식']]  # column 명칭 수정

for name in feature_data.columns.tolist() : 
  cnt = 0
  for i in range(len(feature_data)) : 
    if feature_data[name].isnull()[i] == True : 
      cnt+=1
    elif feature_data[name][i]=='미입사' : 
      cnt+=1
    elif feature_data[name][i]=='미평가' : 
      cnt+=1
  missing_rate[name] = cnt/len(feature_data)

In [ ]:
missing_rate_cnt = {}
for name in feature_data.columns.tolist() : 
  cnt = 0
  for i in range(len(feature_data)) : 
    if feature_data[name].isnull()[i] == True : 
      cnt+=1
    elif feature_data[name][i]=='미입사' : 
      cnt+=1
    elif feature_data[name][i]=='미평가' : 
      cnt+=1
  missing_rate_cnt[name] = cnt

missing_rate_cnt['all'] = len(feature_data)       

In [ ]:
main_dict

In [ ]:
json_file = json.dumps(main_dict)

In [ ]:
file_path = '/content/drive/My Drive/Colab Notebooks/NH농협캐피탈/승진적합_v2/승진적합_다면평가.json'

In [ ]:
with open(file_path, 'w', encoding='utf-8-sig') as file:
    file.write(json.dumps(main_dict, ensure_ascii=False))

In [ ]:
file_path_2 = '/content/drive/My Drive/Colab Notebooks/NH농협캐피탈/output/missing_rate_승진적합(다면평가).json'

In [ ]:
with open(file_path_2, 'w', encoding='utf-8') as file:
    file.write(json.dumps(missing_rate_cnt, ensure_ascii=False))

Inference

In [ ]:
# 수정 1114
inf_x = pd.get_dummies(inf_data.drop(['사번', '승진대상 후보여부'], axis=1), columns=['직책', '성별', '입사경로', '학력', '거주지', '교육 이수여부', '전공'], drop_first=True)
inf_x['직책_팀원'] = 0
inf_x['거주지_대구'] = 0
inf_x['거주지_대전'] = 0
inf_x['교육 이수여부_Yes'] = (inf_x['교육 이수여부_미신청'] != 1).replace({False : 0, True : 1}) 
inf_x = inf_x[X_train.columns]
inf_x = inf_x.astype('float')

In [ ]:
pred = end_model.predict(inf_x)
inf_proba = end_model.predict_proba(inf_x)[:,1]

In [ ]:
result = pd.DataFrame(columns=['승진여부'])
result['승진여부'] = pred
result['predict_proba'] = inf_proba

In [ ]:
result['승진여부'] = result['승진여부'].replace(0, '유지')
result['승진여부'] = result['승진여부'].replace(1, '승진')

In [ ]:
final_result = pd.concat([inf_data, result], axis=1)

In [ ]:
# 수정 1124

for i in range(len(final_result)) : 
  if final_result['predict_proba'][i] < 0.5 : 
    if (final_result['셀 참여횟수'][i]!=0 and final_result['봉사활동 참여횟수'][i]!=0) : 
      final_result['predict_proba'][i] = final_result['predict_proba'][i] + final_result['셀 참여횟수'][i]*0.08 + final_result['봉사활동 참여횟수'][i]*0.08+0.05
    else : 
      final_result['predict_proba'][i] = final_result['predict_proba'][i] + final_result['셀 참여횟수'][i]*0.08 + final_result['봉사활동 참여횟수'][i]*0.08

for i in range(len(final_result)) :
  if (final_result['predict_proba'][i] + final_result['수상횟수'][i]*0.04) < 1 : 
    final_result['predict_proba'][i] = final_result['predict_proba'][i] + final_result['수상횟수'][i]*0.04

punish_list = ['B1210011', 'B1110035', 'B1110036', 'B1110058', 'B1110077', 'B1510023', 'B1510062', 'B1210021', 'B1210076']
punish_data = []
for i in range(len(final_result)) : 
  if final_result['사번'][i] in punish_list : 
    punish_data.append(1)
  elif final_result['사번'][i] == 'B1310064' : 
    punish_data.append(2)
  else : 
    punish_data.append(0)

for i in range(len(final_result)) :
  if (final_result['predict_proba'][i] - punish_data[i]*0.04) > 0 : 
    final_result['predict_proba'][i] = final_result['predict_proba'][i] - punish_data[i]*0.04

for i in range(len(final_result)) : 
  if final_result['predict_proba'][i] >=0.5 : 
    final_result['승진여부'][i] = '승진'

In [ ]:
private_feature = ['성명', '연락처', '이메일', '생년월일', '본부', '실', '팀', '직무', '직급']
for name in private_feature : 
  final_result = final_result.join(data_2022.set_index('사번')[name], on='사번')

In [ ]:
# 변경
final_result = final_result[['사번', '성명', '연락처', '이메일', '생년월일', '본부', '실', '팀', '직무', '직급', '직책', '성별', '연령', '당사근속기간(개월)', '외부경력(개월)', '총경력(개월)', '현 직급 근속기간(연)', '이직횟수',
       '입사경로', '학력', '전공', '거주지', '셀 참여횟수', '봉사활동 참여횟수', '직무관련자격증', '수상횟수', '평균 교육점수',
       '교육 이수여부', '당년 역량평가등급', '당년 업적평가등급', '역량평가등급(1년전)', '역량평가등급(2년전)', '업적평가등급(1년전)', '업적평가등급(2년전)', '전문성_평균평점', '리더십_평균평점', '부하육성_평균평점', '관계관리_평균평점', '조직비전_평균평점',
       '직책수행능력_평균평점', '조직내 필요성_평균평점', '동반자 의식_평균평점', '직책수행능력', '조직내 필요성',
       '동반자 의식', '승진적합 산출점수', '승진여부', 'predict_proba']]  # column 명칭 수정

In [ ]:
# 추가 3년치 평가등급 보여주기
final_result[['평균 교육점수', '당년 역량평가등급', '당년 업적평가등급', '역량평가등급(1년전)', '역량평가등급(2년전)', '업적평가등급(1년전)', '업적평가등급(2년전)']] = data_2022[['평균 교육점수', '당년 역량평가등급', '당년 업적평가등급', '역량평가등급(1년전)', '역량평가등급(2년전)', '업적평가등급(1년전)', '업적평가등급(2년전)']]

In [ ]:
shap_values_inf = explainer.shap_values(inf_x) 

In [ ]:
pos_max = shap_values_inf.max(axis=0)
neg_max = shap_values_inf.min(axis=0)
new_list = []
for row in shap_values_inf:
  temp = []
  for ele, pos, neg in zip(row, pos_max, neg_max):
    if ele >= 0:
      if pos == 0: 
        temp.append(0)
      else:
        temp.append(ele/pos)
    else:
      if neg == 0:
        temp.append(0)
      else:
        temp.append(-ele/neg)
  new_list.append(temp)

feature_weight = pd.DataFrame(new_list, index = pd.RangeIndex(len(shap_values_inf)), columns = X_test.columns)
feature_weight = feature_weight.fillna(0)

In [ ]:
categorical_feature = ['직책', '성별', '입사경로', '학력', '거주지', '교육 이수여부', '전공']
cate_main = []

for i in range(len(feature_weight)) :
  temp_1 = []
  for name in categorical_feature : 
    temp_2 = []
    for j in range (len(feature_weight.columns.tolist())) : 
      if name in feature_weight.columns.tolist()[j] : 
        real_name = feature_weight.columns.tolist()[j]
        temp_2.append(feature_weight[real_name][i])
    data_sum = np.sum(temp_2)
    temp_1.append(data_sum)
  cate_main.append(temp_1)

In [ ]:
ttt = pd.DataFrame(cate_main, columns=categorical_feature)

feature_weight.drop(['직책_실장', '직책_지점장', '직책_팀원', '직책_팀장', '성별_여자',
       '학력_대졸', '학력_대학원졸', '학력_전졸', '거주지_광주', '거주지_대구', '거주지_대전', '거주지_부산',
       '거주지_서울', '거주지_인천', '교육 이수여부_Yes', '교육 이수여부_미신청', '전공_공학계열', '전공_사회계열',
       '전공_자연계열'], axis=1, inplace=True)
feature_weight = pd.concat([feature_weight, ttt], axis=1)

In [ ]:
mm = MinMaxScaler(feature_range=(-1, 1))

In [ ]:
for i in range (len(feature_weight.max())) : 
  if feature_weight.max()[i] > 1 : 
    feature_name = feature_weight.max().index[i]
    scaling = mm.fit_transform(feature_weight[feature_name].values.reshape(-1,1))
    for j in range (len(feature_weight)) : 
      feature_weight[feature_name][j] = scaling[j][0]

for i in range (len(feature_weight.min())) : 
  if feature_weight.min()[i] < -1 : 
    feature_name = feature_weight.min().index[i]
    scaling = mm.fit_transform(feature_weight[feature_name].values.reshape(-1,1))
    for j in range (len(feature_weight)) : 
      feature_weight[feature_name][j] = scaling[j][0]

In [ ]:
feature_weight.insert(0, '사번', inf_data['사번'].tolist())

In [ ]:
# 정성요인 가중치 로직으로 아직 대표님 검토 중
feature_weight['셀 참여횟수'] = final_result['셀 참여횟수']*0.1
feature_weight['봉사활동 참여횟수'] = final_result['봉사활동 참여횟수']*0.1
feature_weight['현 직급 근속기간(연)'] = feature_weight['현 직급 근속기간(연)']*0.1
feature_weight['전공'] = feature_weight['전공']*0.4

In [ ]:
feature_weight.to_csv('/content/drive/My Drive/Colab Notebooks/NH농협캐피탈/승진적합_v2/feature_weight_승진적합_다면평가.csv', encoding='utf-8-sig', index=False)

In [ ]:
# 수정 1116
attrition = ['정희진', '박영민', '강도이', '박상완']
attrition_idx = []
for i in range(len(final_result)) : 
 if final_result['성명'][i] in attrition : 
   attrition_idx.append(i)

final_result.drop(attrition_idx, inplace=True)

In [ ]:
# 수정 1111
final_result.drop(final_result.loc[final_result['직급']=='부장'][final_result.loc[final_result['직급']=='부장']['현 직급 근속기간(연)']<3].index.tolist(), inplace=True)

In [ ]:
# 수정 1110
final_result.drop(final_result.loc[final_result['직급']=='이사대우'].index.tolist(), inplace=True)
final_result.drop(final_result.loc[final_result['직급']=='기획역'].index.tolist(), inplace=True)

In [ ]:
# 수정 1107
final_result.replace({'직급' : ['업무지원역', '기획역']}, '기타', inplace=True)

In [ ]:
final_result
final_result.to_csv('/content/drive/My Drive/Colab Notebooks/NH농협캐피탈/승진적합_v2/output_승진적합_다면평가.csv', encoding='utf-8-sig', index=False)

In [ ]:
qwer = final_result.copy()
qqqq = qwer[numerical]
qqqq['predict_proba'] = qwer['predict_proba']
qqqq = qqqq.reset_index(drop=True)

In [ ]:
grade_list = ['당년 역량평가등급', '당년 업적평가등급', '역량평가등급(1년전)', '역량평가등급(2년전)', '업적평가등급(1년전)', '업적평가등급(2년전)']

for column in grade_list : 
  for i in range(len(qqqq)) : 
    if qqqq[column][i] == 'S' :
      qqqq[column][i] = 5
    elif qqqq[column][i] == 'A' :
      qqqq[column][i] = 4
    elif qqqq[column][i] == 'B' :
      qqqq[column][i] = 3
    elif qqqq[column][i] == 'C' :
      qqqq[column][i] = 2
    elif qqqq[column][i] == 'D' :
      qqqq[column][i] = 1
    else : 
      qqqq[column][i] = 0

In [ ]:
qqqq.replace('미신청', 0, inplace=True)
qqqq.replace('미입사', 0, inplace=True)

In [ ]:
qqqq = qqqq.astype('float')

In [ ]:
ffff= qqqq.corr(method='pearson')

In [ ]:
df = pd.DataFrame(ffff['predict_proba'].sort_values(ascending=False), columns=['predict_proba'])
df.style.background_gradient(cmap='viridis')